## Init Database

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///yago.db')
DBSession = sessionmaker(bind=engine)
db_session = DBSession()

Define tables:

In [2]:
from sqlalchemy import Column, String, Integer, ForeignKey, CHAR
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
import sqlite3

Base = declarative_base()

In [3]:
class Country(Base):
    __tablename__ = 'country'
    
    country_id = Column(String(64), primary_key=True)
    
    def __init__(self, country_id):
        self.country_id = country_id

In [4]:
class University(Base):
    __tablename__ = 'university'
    
    university_id = Column(String(64), primary_key=True)
    
    def __init__(self, university_id):
        self.university_id = university_id

In [5]:
class Politician(Base):
    __tablename__ = 'politician'
    
    politician_id = Column(String(64), primary_key=True)
    gender = Column(CHAR)
    given_name = Column(String(64))
    family_name = Column(String(64))
    
    def __init__(self, politician_id):
        self.politician_id = politician_id

In [6]:
class PoliticianCountry(Base):
    __tablename__ = 'politician_country'

    politician_id = Column(String(64), ForeignKey('politician.politician_id'), unique=False, primary_key=True)
    politician = relationship(Politician)
    country_id = Column(String(64), ForeignKey('country.country_id'), unique=False, primary_key=True)
    country = relationship(Country)
    
    def __init__(self, politician, country):
        self.politician = politician
        self.country = country

In [7]:
class PoliticianUniversity(Base):
    __tablename__ = 'politician_university'

    politician_id = Column(String(64), ForeignKey('politician.politician_id'), unique=False, primary_key=True)
    politician = relationship(Politician)
    university_id = Column(String(64), ForeignKey('university.university_id'), unique=False, primary_key=True)
    university = relationship(University)
    
    def __init__(self, politician, university):
        self.politician = politician
        self.university = university

In [8]:
class Prize(Base):
    __tablename__ = 'prize'
    
    prize_id = Column(String(128), primary_key=True)
    
    def __init__(self, prize_id):
        self.prize_id = prize_id

In [9]:
class PoliticianPrize(Base):
    __tablename__ = 'politician_prize'

    politician_id = Column(String(64), ForeignKey('politician.politician_id'), unique=False, primary_key=True)
    politician = relationship(Politician)
    prize_id = Column(String(64), ForeignKey('prize.prize_id'), unique=False, primary_key=True)
    prize = relationship(Prize)
    
    def __init__(self, politician, prize):
        self.politician = politician
        self.prize = prize

In [10]:
class Website(Base):
    __tablename__ = 'website'
    
    website_id = Column(String(128), primary_key=True)
    politician_id = Column(String(64), ForeignKey('politician.politician_id'), unique=False)
    politician = relationship(Politician)
    
    def __init__(self, website_id, politician):
        self.website_id = website_id
        self.politician = politician

In [11]:
class PoliticalParty(Base):
    __tablename__ = 'political_party'
    
    party_id = Column(String(128), primary_key=True)
    
    def __init__(self, party_id):
        self.party_id = party_id

In [12]:
class PoliticianPoliticalParty(Base):
    __tablename__ = 'politician_political_party'
    
    politician_id = Column(String(64), ForeignKey('politician.politician_id'), unique=False, primary_key=True)
    politician = relationship(Politician)
    party_id = Column(String(64), ForeignKey('political_party.party_id'), unique=False, primary_key=True)
    party = relationship(PoliticalParty)
    
    def __init__(self, politician, party):
        self.politician = politician
        self.party = party

In [13]:
Base.metadata.create_all(engine)

## Add data to database

In [14]:
from neo4j.v1 import GraphDatabase

neo_driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "123"))
neo_session = neo_driver.session()

In [15]:
query_limit = '' # 'LIMIT 500'

Populate `politician`:

In [16]:
def add_politician_country(tx):
    for record in tx.run("MATCH (politician:Entity)-[:isPoliticianOf]->(country:Entity) "
                         "WHERE country.name = \"<Iran>\" "
                         "RETURN politician.name, country.name {}".format(query_limit)):
        country_name = record['country.name']
        politician_name = record['politician.name']
        
        country = db_session.query(Country).filter(Country.country_id == country_name).first()
        if country is None:
            country = Country(country_name)
            db_session.add(country)
    
        politician = db_session.query(Politician).filter(Politician.politician_id == politician_name).first()
        if politician is None:
            politician = Politician(politician_name)
            db_session.add(politician)

        politician_country = PoliticianCountry(politician, country)
        db_session.add(politician_country)

neo_session.write_transaction(add_politician_country)

In [17]:
def add_politician_gender(tx):
    for record in tx.run("MATCH (politician:Entity)-[:isPoliticianOf]->(country:Entity) "
                         "WHERE country.name = \"<Iran>\" "
                         "MATCH (politician)-[:hasGender]->(gender) "
                         "RETURN politician.name, gender.name {}".format(query_limit)):
        gender = record['gender.name'][1:-1]
        politician_name = record['politician.name']
            
        politician = db_session.query(Politician).filter(Politician.politician_id == politician_name).first()
        if politician is None:
            continue
        politician.gender = gender

        db_session.add(politician)

neo_session.write_transaction(add_politician_gender)

In [18]:
def add_politician_university(tx):
    for record in tx.run("MATCH (politician:Entity)-[:isPoliticianOf]->(country:Entity) "
                         "WHERE country.name = \"<Iran>\" "
                         "MATCH (politician)-[:graduatedFrom]->(university:Entity) "
                         "RETURN politician.name, university.name {}".format(query_limit)):
        university_name = record['university.name']
        politician_name = record['politician.name']
 
        politician = db_session.query(Politician).filter(Politician.politician_id == politician_name).first()
        if politician is None:
            continue

        university = db_session.query(University).filter(University.university_id == university_name).first()
        if university is None:
            university = University(university_name)
            db_session.add(university)
            
        politician_university = PoliticianUniversity(politician, university)
        db_session.add(politician_university)

neo_session.write_transaction(add_politician_university)

In [19]:
def add_politician_given_name(tx):
    for record in tx.run("MATCH (politician:Entity)-[:isPoliticianOf]->(country:Entity) "
                         "WHERE country.name = \"<Iran>\" "
                         "MATCH (politician)-[:hasGivenName]->(givenName) "
                         "RETURN politician.name, givenName.name {}".format(query_limit)):
        given_name = record['givenName.name']
        politician_name = record['politician.name']
 
        politician = db_session.query(Politician).filter(Politician.politician_id == politician_name).first()
        if politician is None:
            continue
        politician.given_name = given_name
        
        db_session.add(politician)

neo_session.write_transaction(add_politician_given_name)

In [20]:
def add_politician_family_name(tx):
    for record in tx.run("MATCH (politician:Entity)-[:isPoliticianOf]->(country:Entity) "
                         "WHERE country.name = \"<Iran>\" "
                         "MATCH (politician:Entity)-[:hasFamilyName]->(familyName) "
                         "RETURN politician.name, familyName.name {}".format(query_limit)):
        family_name = record['familyName.name']
        politician_name = record['politician.name']
 
        politician = db_session.query(Politician).filter(Politician.politician_id == politician_name).first()
        if politician is None:
            continue
        politician.family_name = family_name
        
        db_session.add(politician)

neo_session.write_transaction(add_politician_family_name)

In [21]:
def add_politician_prize(tx):
    for record in tx.run("MATCH (politician:Entity)-[:isPoliticianOf]->(country:Entity) "
                         "WHERE country.name = \"<Iran>\" "
                         "MATCH (politician)-[:hasWonPrize]->(prize) "
                         "RETURN politician.name, prize.name {}".format(query_limit)):
        prize_name = record['prize.name']
        politician_name = record['politician.name']
        
        prize = db_session.query(Prize).filter(Prize.prize_id == prize_name).first()
        if prize is None:
            prize = Prize(prize_name)
            db_session.add(prize)
    
        politician = db_session.query(Politician).filter(Politician.politician_id == politician_name).first()
        if politician is None:
            continue

        politician_prize = PoliticianPrize(politician, prize)
        db_session.add(politician_prize)

neo_session.write_transaction(add_politician_prize)

In [22]:
def add_politician_website(tx):
    for record in tx.run("MATCH (politician:Entity)-[:isPoliticianOf]->(country:Entity) "
                         "WHERE country.name = \"<Iran>\" "
                         "MATCH (politician)-[:hasWebsite]->(website) "
                         "RETURN politician.name, website.name {}".format(query_limit)):
        website_name = record['website.name']
        politician_name = record['politician.name']
        
        politician = db_session.query(Politician).filter(Politician.politician_id == politician_name).first()
        if politician is None:
            continue
        
        website = db_session.query(Website).filter(Website.website_id == website_name).first()
        if website is not None:
            continue
        website = Website(website_name, politician)
        db_session.add(website)

neo_session.write_transaction(add_politician_website)

In [23]:
def add_politician_political_party(tx):
    for record in tx.run("MATCH (politician:Entity)-[:isPoliticianOf]->(country:Entity) "
                         "WHERE country.name = \"<Iran>\" "
                         "MATCH (politician)-[:isAffiliatedTo]->(party:Entity) "
                         "MATCH (party)-[:type]->(party_type:Entity) "
                         "WHERE party_type.name = \"<wikicat_Political_parties_in_Iran>\" "
                         "RETURN politician.name, party.name {}".format(query_limit)):
        party_name = record['party.name']
        politician_name = record['politician.name']
        
        party = db_session.query(PoliticalParty).filter(PoliticalParty.party_id == party_name).first()
        if party is None:
            party = PoliticalParty(party_name)
            db_session.add(party)
    
        politician = db_session.query(Politician).filter(Politician.politician_id == politician_name).first()
        if politician is None:
            continue

        politician_party = PoliticianPoliticalParty(politician, party)
        db_session.add(politician_party)

neo_session.write_transaction(add_politician_political_party)

In [24]:
db_session.commit()

In [25]:
db_session.close()